### Imports

In [1206]:
import pandas as pd
import numpy as np
import pickle
from scipy.stats import poisson

### Data Cleaning and Preparation

In [1207]:
df_historical = pd.read_csv("data/afcon_historical_data.csv")
df_fixture = pd.read_csv("data/afcon_fixture_data.csv")
dict_groups = pickle.load(open("data/afcon_groups_dump", "rb"))

##### Cleanup fixture data

In [1208]:
df_fixture["home"] = df_fixture["home"].str.strip()
df_fixture["away"] = df_fixture["away"].str.strip()

In [1209]:
df_fixture.drop(columns=["score"], axis=1, inplace=True)

In [1210]:
df_fixture.rename(columns={"home": "home_team", "away": "away_team"}, inplace=True)

##### Cleanup historical data

In [1211]:
df_historical.dropna(inplace=True)
df_historical.drop_duplicates(inplace=True)
df_historical.sort_values(by="year", inplace=True)

In [1212]:
# remove cancelled games from historical data
df_historical = df_historical[df_historical["score"] != "Cancelled"]

In [1213]:
# split score into home and away goals
df_historical[["home_goals", "away_goals"]] = df_historical["score"].str.split("–", expand=True)
df_historical = df_historical.astype({"home_goals": "int64", "away_goals": "int64"})
df_historical.drop(columns=["score"], axis=1, inplace=True)

In [1214]:
df_historical.rename(columns={"home": "home_team", "away": "away_team"}, inplace=True)

In [1215]:
df_historical["total_goals"] = df_historical["home_goals"] + df_historical["away_goals"]

##### Export clean data

In [1216]:
df_historical.to_csv("data/afcon_historical_data_clean.csv", index=False)
df_historical.to_csv("data/afcon_fixture_data_clean.csv", index=False)

In [1217]:
df_historical

,home_team,away_team,year,home_goals,away_goals,total_goals
0,Sudan,Egypt,1957,1,2,3
1,Ethiopia,South Africa,1957,2,0,2
2,Egypt,Ethiopia,1957,4,0,4
3,United Arab Republic,Ethiopia,1959,4,0,4
4,Sudan,Ethiopia,1959,1,0,1
...,...,...,...,...,...,...
714,Nigeria,Sudan,2021,3,1,4
715,Guinea-Bissau,Egypt,2021,0,1,1
716,Guinea-Bissau,Nigeria,2021,0,2,2
718,Algeria,Sierra Leone,2021,0,0,0


In [1218]:
df_fixture.sample(10)

,home_team,away_team,year
22,Angola,Burkina Faso,2023
9,Cape Verde,Mozambique,2023
51,Winner SF1,Winner SF2,2023
20,Algeria,Burkina Faso,2023
6,Egypt,Mozambique,2023
29,Namibia,Mali,2023
0,Ivory Coast,Guinea-Bissau,2023
17,Gambia,Cameroon,2023
12,Senegal,Gambia,2023
3,Ivory Coast,Nigeria,2023


### Poisson Setup

In [1219]:
# create a dataframe for home and away teams
df_home = df_historical[["home_team", "home_goals", "away_goals"]].copy()
df_away = df_historical[["away_team", "home_goals", "away_goals"]].copy()

In [1220]:
# rename columns to match the fixture data
df_home.rename(columns={"home_team": "team", "home_goals": "goals_for", "away_goals": "goals_against"}, inplace=True)
df_away.rename(columns={"away_team": "team", "home_goals": "goals_against", "away_goals": "goals_for"}, inplace=True)

In [1221]:
df_home["team"] = df_home["team"].str.strip()
df_away["team"] = df_away["team"].str.strip()

In [1222]:
# swap goals for and against for away team
df_away = df_away[["team", "goals_for", "goals_against"]]

In [1223]:
df_away.sample(5)

,team,goals_for,goals_against
55,Sudan,2,1
613,Guinea-Bissau,1,2
25,Tunisia,2,3
159,Togo,1,4
198,Kenya,0,3


In [1224]:
# create a dataframe that calculates the average goals for and against each team
df_team_goals = pd.concat([df_home, df_away], ignore_index=True).groupby(["team"]).mean()

In [1225]:
df_team_goals.sample(5)

,goals_for,goals_against
team,,
United Arab Republic,3.000000,1.571429
Ethiopia,1.064516,2.161290
United Arab Rep.,2.000000,1.000000
Senegal,1.181818,0.833333
Equatorial Guinea,0.733333,0.933333


In [1226]:
df_team_goals.at["Egypt", "goals_for"]

1.4421052631578948

##### Function to calculate poisson distribution for each team

In [1227]:
def predict_points(home, away):
    if home not in df_team_goals.index or away not in df_team_goals.index:
        return (0, 0)    

    # calculate the expected number of goals for each team
    lamb_home = df_team_goals.at[home,"goals_for"] * df_team_goals.at[away,"goals_against"]
    lamb_away = df_team_goals.at[away,"goals_for"] * df_team_goals.at[home,"goals_against"]
    
    # calculate the probability of each score
    x = np.arange(0, 11)
    y = np.arange(0, 11)
    p = np.outer(poisson.pmf(x, lamb_home), poisson.pmf(y, lamb_away))

    # calculate the probability of each result
    prob_draw = np.sum(np.diag(p))
    prob_home = np.sum(np.tril(p, -1))
    prob_away = np.sum(np.triu(p, 1))
    
    # calculate the expected points for each team
    points_home = 3 * prob_home + prob_draw
    points_away = 3 * prob_away + prob_draw
    
    return (points_home, points_away)

In [1228]:
predict_points("Egypt", "Algeria")

(1.8812853734934467, 0.8775671115267686)

### Preparing Fixture Data

In [1229]:
# breaking up the fixture data into stages
df_fixture_group = df_fixture[:36].copy()
df_fixture_knockout = df_fixture[36:44].copy()
df_fixture_quarter = df_fixture[44:48].copy()
df_fixture_semi = df_fixture[48:50].copy()
df_fixture_third = df_fixture[50:51].copy()
df_fixture_final = df_fixture[51:].copy()

In [1230]:
df_fixture_group.sample(5)

,home_team,away_team,year
25,Mali,South Africa,2023
14,Senegal,Cameroon,2023
26,Tunisia,Mali,2023
10,Mozambique,Ghana,2023
1,Nigeria,Equatorial Guinea,2023


#### Group stage

In [1231]:
# update points function
def update_points(row, group):
    home, away = row["home_team"], row["away_team"]
    points_home, points_away = predict_points(home, away)

    # add points to home and away teams
    dict_groups[group].loc[dict_groups[group]["Team"] == home, "Pts"] += points_home
    dict_groups[group].loc[dict_groups[group]["Team"] == away, "Pts"] += points_away

In [1232]:
# update points for group stage
for g in dict_groups:
    teams = dict_groups[g]["Team"].values
    # get the matches for only the teams in the group
    df_fixture_matches = df_fixture_group[df_fixture_group["home_team"].isin(teams)]
    df_fixture_matches.apply(lambda row: update_points(row, g), axis=1)

    # sort the teams by points and round
    dict_groups[g] = dict_groups[g].sort_values("Pts", ascending=False).reset_index(drop=True)
    dict_groups[g] = dict_groups[g][["Team", "Pts"]].round(0)

In [1233]:
dict_groups["Group B"]

,Team,Pts
0,Egypt,6.0
1,Ghana,6.0
2,Cape Verde,4.0
3,Mozambique,1.0


#### Prepare Knockout Stage

In [1234]:
df_fixture_knockout

,home_team,away_team,year
36,Winner Group D,3rd Group B/E/F,2023
37,Runner-up Group A,Runner-up Group C,2023
38,Winner Group A,3rd Group C/D/E,2023
39,Runner-up Group B,Runner-up Group F,2023
40,Winner Group B,3rd Group A/C/D,2023
41,Winner Group C,3rd Group A/B/F,2023
42,Winner Group E,Runner-up Group D,2023
43,Winner Group F,Runner-up Group E,2023


In [1235]:
# replace all the winners and runners up with the teams that qualified
for g in dict_groups:
    winner = dict_groups[g].loc[0, "Team"]
    second = dict_groups[g].loc[1, "Team"]
    df_fixture_knockout.replace({f"Winner {g}": winner,
                                f"Runner-up {g}": second},
                                inplace=True)

In [1236]:
# replace all the 3rd placed teams with the teams that qualified
for group_str in ["3rd Group B/E/F", "3rd Group C/D/E", "3rd Group A/C/D", "3rd Group A/B/F"]:
    group_ids = group_str.split()[2].split("/")
    third_placed_teams = pd.concat([dict_groups[f"Group {g}"].loc[[2], ["Team", "Pts"]] for g in group_ids])
    third_placed_teams.sort_values("Pts", ascending=False, inplace=True)

    # replace the first team that hasn"t already qualified
    for i in range(len(third_placed_teams)):
        if third_placed_teams.iloc[i, 0] not in df_fixture_knockout.values:
            df_fixture_knockout = df_fixture_knockout.replace({group_str: third_placed_teams.iloc[i, 0]})
            break

In [1237]:
df_fixture_knockout["winner"] = np.nan

In [1238]:
df_fixture_knockout

,home_team,away_team,year,winner
36,Algeria,Mali,2023,NaN
37,Nigeria,Senegal,2023,NaN
38,Ivory Coast,Burkina Faso,2023,NaN
39,Ghana,Zambia,2023,NaN
40,Egypt,Equatorial Guinea,2023,NaN
41,Cameroon,Cape Verde,2023,NaN
42,Tunisia,Angola,2023,NaN
43,Morocco,South Africa,2023,NaN


##### Get winner function (based on predict_points)

In [1239]:
def get_winner(df_fix):
    for i, r in df_fix.iterrows():
        # predict points for the home and away team and set winner to the team with the most points
        home, away = r["home_team"], r["away_team"]
        points_home, points_away = predict_points(home, away)
        df_fix.loc[i, "winner"] = home if points_home > points_away else away

    return df_fix

##### Update table function

In [1240]:
def update_table(df_fix_prev, df_fix_next, R):
    for c, (i, _) in enumerate(df_fix_prev.iterrows(), start=1):
        winner = df_fix_prev.loc[i, "winner"]
        df_fix_next.replace({f"Winner {R}{c}": winner}, inplace=True)
    
    df_fix_next["winner"] = np.nan
    return df_fix_next

##### Update and predict function

In [1241]:
def update_and_predict(df_fix_prev, df_fix_next, R):
    df_fix_next = update_table(df_fix_prev, df_fix_next, R)
    df_fix_next = get_winner(df_fix_next)
    return df_fix_next

### Predicting AFCON

#### Qualifiers

In [1242]:
# create a df for all the qualifiers for the round of 16 and their group
teams = pd.concat([df_fixture_knockout["home_team"], df_fixture_knockout["away_team"]])
groups = teams.map({team: group for group, df in dict_groups.items() for team in df["Team"]})

In [1253]:
df_ro16_qualifiers = pd.DataFrame({"Team": teams, "Group": groups})
df_ro16_qualifiers.sort_values("Group", inplace=True)

In [1254]:
df_ro16_qualifiers

,Team,Group
37,Nigeria,Group A
38,Ivory Coast,Group A
40,Equatorial Guinea,Group A
39,Ghana,Group B
40,Egypt,Group B
41,Cape Verde,Group B
41,Cameroon,Group C
37,Senegal,Group C
36,Algeria,Group D
38,Burkina Faso,Group D


#### Round of 16

In [1244]:
# get the winner for the knockout stage
get_winner(df_fixture_knockout)

,home_team,away_team,year,winner
36,Algeria,Mali,2023,Algeria
37,Nigeria,Senegal,2023,Senegal
38,Ivory Coast,Burkina Faso,2023,Ivory Coast
39,Ghana,Zambia,2023,Ghana
40,Egypt,Equatorial Guinea,2023,Egypt
41,Cameroon,Cape Verde,2023,Cameroon
42,Tunisia,Angola,2023,Tunisia
43,Morocco,South Africa,2023,Morocco


#### Quarter Finals ####

In [1245]:
update_and_predict(df_fixture_knockout, df_fixture_quarter, "R")

,home_team,away_team,year,winner
44,Ghana,Ivory Coast,2023,Ghana
45,Senegal,Algeria,2023,Senegal
46,Tunisia,Cameroon,2023,Cameroon
47,Egypt,Morocco,2023,Egypt


#### Semi Finals

In [1246]:
update_and_predict(df_fixture_quarter, df_fixture_semi, "QF")

,home_team,away_team,year,winner
48,Ghana,Egypt,2023,Egypt
49,Cameroon,Senegal,2023,Cameroon


#### Third Place Play-Off

In [1252]:
# create a copy of the semi final fixtures for the 3rd place playoff
df_losers = df_fixture_semi.copy()
df_losers["loser"] = np.where(df_losers["home_team"] == df_losers["winner"], df_losers["away_team"], df_losers["home_team"])

In [1248]:
df_fixture_semi_loser = pd.DataFrame({
    "home_team": [df_losers["loser"].iloc[0]],
    "away_team": [df_losers["loser"].iloc[1]],
    "year": [df_losers["year"].iloc[0]],
    "winner": np.nan
})

In [1249]:
get_winner(df_fixture_semi_loser)

,home_team,away_team,year,winner
0,Ghana,Senegal,2023,Ghana


#### Finals

In [1250]:
update_and_predict(df_fixture_semi, df_fixture_final, "SF")

,home_team,away_team,year,winner
51,Egypt,Cameroon,2023,Egypt


#### Winner !!!

In [1251]:
df_fixture_final["winner"]

51    Egypt
Name: winner, dtype: object